## EXAMEN : scrapping de l'ensemble des produits du site OPENFOODFACTS

https://fr.openfoodfacts.org/

### Présenté par : 
| Prénoms       |     Nom         |   
| ------------- |: -------------: |
| Amadou lamarana      | DIALLO               |


Ce livrable constitue un véritable exercice sur l'extraction des données avec le langage Python. Nous avons pour ce faire utilisé les libraries essentielles que sont:
* **Pandas**: libraire pour la gestion et la manipulation des données **cvs** récupérées
* **BeautifulSoup**:librairie pour l'extraction des fichiers HTML et XML
* **re**: Librairie des expressions regulières
* **requests**: pour l'interrogation des **url**  (requêtes HTTP)
* **unicodedata**: Module de conversion de caractères spéciaux au format **utf-8**

À travers ce notebook , nous allons extraire les liens des produits qui constituent les données de bases
du présent projet.
Un petit aperçu du site montre que le nombre de page est assez grand plus **7980 pages**<br/>
Nous allons récuperer en suivant les étapes citées ci-après:

  ### <div class='text text-primary'>Procédures de récupération des données </div>
1. Récupérer les urls qui pointe sur toutes ces pages
    * Définition des fonctions pour récupérer les pages du site
2. Récuperer les urls des produits contenu dans chacune des pages
    * Définition de la fonction qui recupère les urls de produits d'une page
    * Sauvegarder les liens dans un fichier au format **csv**
3. Récupération des informations des produits
4. Création du Dataframe et l'export sous CSV

<div class='text text-primary' style='background:green; color:white; font-size:16px; padding:26px'> 1. Récupérer les liens des pages </div>

### Importation des modules nécessaires

In [40]:
import re
import requests
from bs4 import BeautifulSoup
## Opendata
## import des modules nécessaires
import pandas as pd 
import unicodedata
from os import path
pd.set_option("max_colwidth", 100)

### Constances

In [41]:
BASE_URL = "https://fr.openfoodfacts.org" ## endpoind du site

In [42]:
PRODUCT_URL_FILE ='url_product.csv' ## fichier des liens des produits

In [43]:
PAGE_URL_FILE ='url_page.csv' ## fichier des liens des pages 

In [44]:
DATA_FILE ='OFF_DATA.csv' ## fichier contenant les données extraites

In [45]:
STATUS_FILE ='OFF_STATUS.csv' ## fichier état et avancement du scraping

<div class='text text-primary' style='background:#0350F0; color:white; font-size:15px; padding:20px'>Fonction qui récupere les pages</div>

**Nous allons procéder par deux méthodes**

    1. Par lecture directe et génération des liens
    2. Par lecture d'un fichier de sauvegarde des liens des pages
    
Nous effectuerons des mesures de performaces et de rapide pour choisir la meilleure

<div class='text' style='background:#000000; color:white; font-size:15px; padding:20px'>1 ere méthode : Lecture directe sur le site à chaque exécution</div>

In [39]:
def recuperer_urls_page():
    """
        * Cette fonction retourne la liste des pages du site 
        * Cette liste initialisé avec l'url de valeur BASE_URL
        * Une requête html avec ** request.get(BASE_URL)** pour tenter de récuperer le contenu de l'accueil
        * Si la page est accessible, une série de manipulation avec BeautifoulSoup sera déclenchée pour 
        * suivant le format de liens, BASE_URL/n avec n le numéro de page, nous allons itérer 
            en tenant compte du numéro de la dernière page disponible sur le site.
    @return list: urls
    """
    #Initiate urls 
    urls= [BASE_URL] ## url de base initialisé avec le lien d'accueil du site (lien de base)
    accueil = requests.get(BASE_URL) ## request html 
    if accueil.status_code == requests.codes.ok: ## All is OK (codeResponse = 200)
        ## parser html
        bs = BeautifulSoup(accueil.text, 'html.parser') 
        ## la pagination est accessible par la balise 'ul' et par l'identifiant "pages"
        ulPages = bs.find('ul',id='pages')
        ## récupérer les éléments de la pagination avec la balise 'li'
        page = ulPages.findAll('li', class_='')
        ## extrait avec le module :'re' des numéros de pages qui seront stockés dans une liste "numeros"
        numeros = [ int(n.text) for n in page if n.text in re.findall('[0-9]*', n.text)]
        derniere_page = max(numeros)
        for i in range(2,derniere_page+1):
            url = BASE_URL+'/'+str(i)
            urls.append(url)
    return urls

<div class='text text-primary' style='background:#FF50F0; color:white; font-size:15px; padding:20px'>Temps d"éxecution: 2.46 s </div>

In [8]:
%%time
methode_1_urls_page = recuperer_urls_page()

CPU times: user 109 ms, sys: 25.5 ms, total: 134 ms
Wall time: 2.27 s


<div class='text' style='background:#000000; color:white; font-size:15px; padding:20px'>2 ieme méthode: Lecture des liens sur un fichier de sauvegarde</div>

In [9]:
def recuperer_lien_page():
    
    """
        * Cette fonction retourne la liste des pages du site 
        * Cette liste initialisé avec l'url de valeur BASE_URL
        * Une requête html avec ** request.get(BASE_URL)** pour tenter de récuperer le contenu de l'accueil
        * Si la page est accessible, une série de manipulation avec BeautifoulSoup sera déclenchée pour 
        * suivant le format de liens, BASE_URL/n avec n le numéro de page, nous allons itérer 
            en tenant compte du numéro de la dernière page disponible sur le site.
        @return list: urls
    """
    accueil = requests.get(BASE_URL) ## request html 
    
    if accueil.status_code == requests.codes.ok: ## All is OK (codeResponse = 200)
        ## parser html
        
        bs = BeautifulSoup(accueil.text, 'html.parser') 
        
        ## la pagination est accessible par la balise 'ul' et par l'identifiant "pages"
        
        pagination = bs.find('ul',id='pages')
        
        ## récupérer les éléments de la pagination avec la balise 'li'
        
        pages = pagination.findAll('li', class_='')
        
        ## extrait avec le module :'re' des numéros de pages qui seront stockés dans une liste "numeros"
        
        numeros = [ int(n.text) for n in pages if n.text in re.findall('[0-9]*', n.text)]
        
        ## Vérifier s'il y déja un fichier des urls produits en csv existant
        if path.exists(PAGE_URL_FILE):
            try:
                ## urls nouveaux à concatener aux urls existants dans le fichier 
                nouveaux_urls= []
                ## le fichier existe et n'est pas vide
                DF = pd.read_csv(PAGE_URL_FILE, engine='python')
                ## on recupère le nombre de page enregistrées dans le fichier csv
                nombre_page  = DF.shape[0]
                ## nombre de page traité
                nombre_page_non_traite=  max(numeros) -nombre_page
                
                if nombre_page_non_traite >0:
                    
                    ## s'il y a de nouvelles pages . On effectue une boucle de la page suivante du dernier url du
                    ## du dataset PAGE_URL_FILE jusqu'a la fin
                    
                    for i in range(nombre_page+1, max(numeros)+1):
                        
                        ## on génère le lien // aka: BASE_URL + "/"+i
                        
                        nouveau_url = BASE_URL+'/'+str(i)
                        
                        ## on ajoute à la liste
                        
                        nouveaux_urls.append(nouveau_url)
                        
                    ## en fin de boucle nous avons les nouveaux liens depuis la dernière mise à jour 
                    ## des liens des pages
                    
                    ## on effectue l'ajout des nouveaux urls sur la liste déja disponble
                    nouveau_DF = pd.DataFrame(nouveaux_urls)
                    nouveau_DF.columns=['url_page']
                    
                    #list_urls_disponible = list(DF['url_page'])
                    
                    ## concaténation 
                    #list_page_urls = list_page_urls + nouveaux_urls
                    newDF = pd.concat([DF, nouveau_DF], ignore_index=True)
                    
                    ## on génére un nouveau fichier csv pour une sauvegarde et base d'une prochaine vérification
                    #newDF = pd.DataFrame(list_page_urls)
                    #newDF.columns =['url_page']
                    newDF.to_csv(PAGE_URL_FILE, index=False , encoding='utf-8')
                    
            except pd.errors.EmptyDataError:
                print('File empty')
        else:
            #Initialiser la liste des liens 
    
            urls= [BASE_URL] 
            ## effectuer une boucle de la deuxieme page jusqu'a la fin
            
            for i in range(2, max(numeros)+1):
                ## ajouter le numero de la page sur le lien de la page
                url = BASE_URL+'/'+str(i)
                ## ajouter le lien de la page sur la liste des urls
                urls.append(url)
                
            ## une  fois la boucle terminée , nous avons une liste de toutes les pages .. à savugarder 
            DF = pd.DataFrame(urls)
            
            ## on attribue un nom à la colonne
            DF.columns = ['url_page']
            
            ## on sauvegarde le dataFrame en fichier csv pour une prochaine utilisation
            DF.to_csv(PAGE_URL_FILE, index=False, encoding = 'utf-8')
            
    ## en appelant cette fonction on renvoie la DataFrame du fichier stocké ("url_page.csv")
    return pd.read_csv(PAGE_URL_FILE, engine='python')

<div class='text text-primary' style='background:#635FF0; color:white; font-size:15px; padding:20px'>Génerer le dataframe des pages</div>

Nous générons notre DataFrame pandas de la liste de pages du site ..  et nous notons une légère amélioration.

<div class='text text-primary' style='background:#09FF90; color:white; font-size:19px;font-weight:bold; padding:20px'>Temps d"éxecution: 2.14 s </div>

In [10]:
%%time
DF_Page_url = recuperer_lien_page()

CPU times: user 152 ms, sys: 13.4 ms, total: 165 ms
Wall time: 2.18 s


In [19]:
DF_Page_url

,url_page
0,https://fr.openfoodfacts.org
1,https://fr.openfoodfacts.org/2
2,https://fr.openfoodfacts.org/3
3,https://fr.openfoodfacts.org/4
4,https://fr.openfoodfacts.org/5
...,...
8073,https://fr.openfoodfacts.org/8074
8074,https://fr.openfoodfacts.org/8075
8075,https://fr.openfoodfacts.org/8076
8076,https://fr.openfoodfacts.org/8077


## <div class='text text-primary' style='background:#0350F0; color:white; font-size:15px; padding:20px'>Étape 2: Récuperer les liens des produits</div>

**Comme la méthode précedente, nous procéderons d'abord par un chargement à partir d'une extraction sur le site et ensuite la méthode de lecture à partir d'un fichier et effectuer des modifications si notées sur le site**


On définit deux fonctions :

    + recuperer_url_produits_page(url_page): retourne les liens des produits dans une page
    
    + recuperer_urls_produits_site(urls_pages)

<div class='text' style='background:#000000; color:white; font-size:15px; padding:20px'>1 iere méthode: Lecture et chargement suite extraction sur le site</div>

In [20]:
def recuperer_url_produits_page(url_page):
    """Fonction qui renvoie les urls des produits listé dans la page du lien passé en paramètre"""
    page = requests.get(url_page)
    if page.status_code == requests.codes.ok:
        bs = BeautifulSoup(page.text, 'html.parser')
        bloc_produits = bs.find('ul', class_ ='products')
        #print(bs)
        urls_produits =[]
        les_produits = bloc_produits.findAll('li',class_='')
        if les_produits is not None and les_produits != []:
            urls_produits = [BASE_URL+url.find("a")['href'] for url in les_produits if url is not None]
        return urls_produits
    else:
        return None

  #####  ETAPE 2. 2  Recupérer les urls de tous des produits de toute les pages 

In [37]:
def recuperer_urls_produits_site(urls_pages):
    """Focntion qui récupére les liens des produits du site
        * Nous utilisons la fonction "recuperer_urls_page" pour avoir les liens des pages;
        * Nous utilisons la fonction "recuperer_url_produits_page"  pour récupérer 
        les liens des produits par page
        @return list: urls_produits
    """
    urls_produits =[]
    i = 0
    ## les produits par page
    for page in urls_pages:
        i= i+1
        if i%100 ==0:
            print("Page :",i)
        #time.sleep(random.uniform(0.2,0.5))
        lien_produits = recuperer_url_produits_page(page)
        if lien_produits is not None and lien_produits !=[]:
            urls_produits = urls_produits + lien_produits
    return urls_produits

<div class='text text-primary' style='background:#FF0000; color:white; font-size:15px; padding:20px'>Temps d"éxecution: xx minutes </div>

Nous n'allons pas exécuter cette requête chaque fois que cela est nécessaire .. Nous allons optimiser en definissant une fonction qui utilise le résultat d'une première exécution (qui est indispensable)

In [ ]:
%%time
## recuperer les urls des produits, enregistrer à la variable urls_produits
urls_produits_site = recuperer_urls_produits_site(methode_1_urls_page)

## <div class='text text-primary' style='background:#FF0000; color:white; font-size:15px; padding:20px'>Cette méthode prends bcp de temps à récuperer les données de liens des produits. Plus de 5h pour extraire juste les liens des produits</div>

## <div class='text text-primary' style='background:#FF0000; color:white; font-size:15px; padding:20px'>Une optimisation est nécessaire pour l'extraction ds données du site en entier .<br>
D'abord nous allons améliorer en utilisant un fichier de sauvegade. Ensuite Nous utiliserons Scrappy qui est plus rapide et plus efficace et plus performant.</div>

> On va procéder de la même manière que lors de la génération des liens des pages .. 
> <b>On part d'un fichier des urls des produits existant où nous recupérons le nombre de liens enregistrés et procéder au complément, suivant le nombre de page disponible.</b> <br/>
> <b>Sinon si le fichier n'existe pas, il faut le créer </b>

In [46]:
def optimiser_recuperer_urls_produit(df_page):
    """
    Cette fonction renvoie les liens des produits à partir du dataFrame des urls des pages en argument
    et retourne un dataframe pandas des liens.
    """
    ## Nous partons de l'existance ou non du fichier des liens des produits
    ## s'il existe c'est une mise à jour 
    ## sinon, c'est une création avec les liens de tous les produits.
    
    ## verifier le type du paramètre // aka il faut un dataframe. sinon KO
    if isinstance(df_page, pd.DataFrame):
        ## Si mise à jour (si le fichier url_product.csv existe)
        if path.exists(PRODUCT_URL_FILE):
            try:

                ## on lit le fichier csv
                DF = pd.read_csv(PRODUCT_URL_FILE, engine='python')
                ## on determine le nombre de lien enregistrés dans le fichier
                nombre_liens_produits = DF.shape[0] ## 80
                
                ## nombre de pages 
                nombre_page = df_page.shape[0] ## 1
                
                ## determiner le nombre de pages traité et le mobre de lien de la derniere page
                nombre_page_traite = nombre_liens_produits//100 ## ->0+1
                
                ## nombre de liens produits traité de la derniere page
                ## le reste de divion par 100 
                nombre_liens_produit_derniere_page = nombre_liens_produits%100
                
                ## on verifie s'il y a de mettre à jour .. avec obligation de vérfier le la derniere page s'il y
                ## ajout de nouveau produit. aka. le nombre de page peut ne pas changé et que le nombre de produit de la 
                ## derniere page change
                
                if nombre_page_traite == nombre_page:
                    print('Derniere déja page traitée:\n')
                    print('Vérification de changement sur la derniere page..:\n')
                    ## Mise à jour des urls de la derniere page. 
                    ## juste le nombre de produit de la dernière page aurait peut-être changé 
                    page = requests.get(df_page.iloc[-1,:]['url_page'])
                    
                    ## si la page est accessible
                    if page.status_code == requests.codes.ok:
                        ## parser le resultat en html
                        bs = BeautifulSoup(page.text, 'html.parser')
                        ## intexer le bloc de la balise 'ul' ayant la classe 'products' 
                        bloc_produits = bs.find('ul',class_='products')
                        ## liste des produits de la page
                        les_produits = bloc_produits.findAll('li',class_='')
                        ## verifier si le nombre a changé, (c'est dire s'il de nouveaux produits dans la page)
                        
                        if nombre_liens_produit_derniere_page:
                            ## le nombre de produits de la page a changé
                            
                            ## recupération des nouveaux urls et sauvegarder dans une liste nouveaux_urls_produits
                            ## faire une boucle des sur les produits et récupérer les liens 
                            nouveaux_urls_produits = [BASE_URL+url.find("a")['href'] for url in les_produits[nombre_liens_produit_derniere_page+1:]]
                            
                            ##creation d'un dataframe
                            newdf_urls_produits = pd.DataFrame(nouveaux_urls_produits)
                            newdf_urls_produits.columns = ['url_product']
                            
                            ## concatenation avec le dataframe originale
                            print("Création du dataframe..\n")
                            DF = pd.concat([DF, newdf_urls_produits], ignore_index=True)
                            
                            ## sauvegarder la derniere version 
                            DF.to_csv(PRODUCT_URL_FILE, index=False, encoding = 'utf-8')
                            print("Changement sur la derniere page integrée..\n")
                            ## retourner le dataFrame
                            return DF
                        else:
                            ## si pas de changement sur la derniere page .. on renvoie le DF non changé
                            print("Pas de changement sur le site fr.openfoodfact.org .......\n")
                            print("On retourne le dataframe URL des produits........\n")
                            return DF
                elif nombre_page_traite  < nombre_page:
                    ## si le nombre de page  du dataframe des urls pages est supérieure
                    print("Le nombre de page récupérée est supérieure au au nombre de page antérieure..")
                    print("Une mise à jour oblige... \n")
                    n_rows = nombre_page_traite *100
                    
                    ##  on reduit le nombre de ligne .. et ignore les dernieres
                    DF = DF.iloc[0:n_rows,:]
                    
                    ##recupérer les liens de page suivante jusqu'a la fin
                    page_restante = list(df_page.iloc[nombre_page_traite+1:,:]['url_page'])
                    
                    nouveaux_urls_produits =[] ## liste pour sauvegarder les liens nouveaux
                    for url_page in page_restante:
                        
                        page = requests.get(url_page)
                        if page.status_code == requests.codes.ok:
                            
                            bs = BeautifulSoup(page.text, 'html.parser')
                            bloc_produits = bs.find('ul',class_='products')
                            ## liste des produits de la page
                            les_produits = bloc_produits.findAll('li',class_='')
                           
                            nouveaux_urls_produits = nouveaux_urls_produits +[BASE_URL+url.find("a")['href'] for url in les_produits]
                            
                    ##creation d'un dataframe
                    newdf_urls_produits = pd.DataFrame(nouveaux_urls_produits)
                    newdf_urls_produits.columns = ['url_product']
                    ## concatenation avec le dataframe originale
                    DF = pd.concat([DF, newdf_urls_produits], ignore_index=True)
                            
                    ## sauvegarder la derniere version 
                    DF.to_csv(PRODUCT_URL_FILE, index=False, encoding = 'utf-8')
                    #print(DF)
                    #newDF = 
                    ## faire une boucle sur les pages non traité et récuperer les urls des produits.
                    
                    return DF
            except pd.errors.EmptyDataError:
                print("Impossible d'ouvrir le fichier")
                
        else:
            ## le fichier n'existe pas .. donc on le crée sous format csv
            print("Le fichier csv n'existe pas .. nous allons le créer \n")
            nouveaux_urls_produits =[]
            for url_page in list(df_page['url_page']):
                page = requests.get(url_page)
                if page.status_code == requests.codes.ok:
                    bs = BeautifulSoup(page.text, 'html.parser')
                    bloc_produits = bs.find('ul',class_='products')
                    ## liste des produits de la page
                    les_produits = bloc_produits.findAll('li',class_='')

                    nouveaux_urls_produits = nouveaux_urls_produits +[BASE_URL+url.find("a")['href'] for url in les_produits]
            
            print("Création du dataframe......... \n")
            DF =  pd.DataFrame(nouveaux_urls_produits)
            print("Sauveagder du dataframe......... \n")
            DF.to_csv(PRODUCT_URL_FILE, index=False, encoding = 'utf-8')
            
            return DF
    else:
        ## l'objet n'est pas un dataframe.. 
        print("Ce n'est pas un DataFrame")
    

In [47]:
%%time
Df_Url_produit = optimiser_recuperer_urls_produit(DF_Page_url)

Le nombre de page récupérée est supérieure au au nombre de page antérieure..
Une mise à jour oblige... 

CPU times: user 9.25 s, sys: 1.74 s, total: 11 s
Wall time: 21.4 s


<div class='text text-primary' style='background:#FF0000; color:white; font-size:15px; padding:20px'>Nous avons un dataframe de liens de produits de 807200 lignes</div>

In [24]:
Df_Url_produit

807640

In [25]:
df_urls_produits_site = pd.read_csv('url_product.csv')
urls_produits = list(df_urls_produits_site['url_product'])

####  Etape 3 : Fonction de récupération des informations des produits 

In [26]:
### On part du fichier csv des url des produits..

df_urls_produits_site = pd.read_csv('url_product.csv')
urls_produits = list(df_urls_produits_site['url_product'])

# ####  Etape 3 : Recupération des informations des produits 
len(urls_produits)


def supprimer_espaces_carcteres_speciaux(text):
    """Supprime les retours à la ligne et normalise le texte, 
    remplace les tabulations, l'apostrphe """
    if text is not None:
        text = text.replace('\t','').replace("'",'_').replace('\n','').replace('"','_').strip().lower()
        return unicodedata.normalize("NFKD", text)
    else:
        return 'XXX'
    #return unicodedata.normalize("NFKD", text.replace('\n',' ').replace('\t',' ').replace("'",'_').strip().lower())
    
def convertible(text):
    chiffres = ["0","1","2","3","4","5","6","7","8","9"]
    pattern = re.compile(r'[+-]?([0-9]*[.])?[0-9]+$')
    if text=='XXX':
        return 'XXX'
    elif (text[0] not in chiffres) and len(text)>1:
        #print(text[1:])
        if (text[0] == '-') and ("%" in text):
            return 'XXX'
        else:
            text = text[1:].replace(',','.').strip()
            test = re.match(pattern, text)
            if test:
                return float(text)
            else:
                return 'XXX'
    elif text[0] in chiffres:
        try:
            test = re.match(pattern, text)
            if test:
                return float(text)
            else:
                return 'XXX'
        except:
            return 'XXX'
    else:
        return "XXX"

## function qui renvoie le nom
def recuperer_nom(product_tag):
    """Renvoie le nom s'il existe sinon XXX"""
    nom_produit='XXX'
    if product_tag:
        nom_brut = product_tag.find('h1',itemprop='name')
        if nom_brut:
            nom_produit = nom_brut.get_text(strip=True).strip()
    return nom_produit

def recuperer_code_barre(product):
    """Renvoie le code barre du produit"""
    code_barre='XXX'
    if product:
        code_barre_tag = product.find('span',id="barcode", itemprop="gtin13")
        if code_barre_tag:
            code_barre = code_barre_tag.text.strip()

    return code_barre

def recuperer_nutriscore(product_tag):
    """Renvoie le nutri-score"""
    nutri_score ='XXX'
    if product_tag:
        score_brut = product_tag.find('div',id='nutriscore_drop') 
        if score_brut:
            nutri_score = supprimer_espaces_carcteres_speciaux(score_brut.find_all('p')[-1].text.split(':')[-1]).upper()
        
    return nutri_score

def recuperer_nova(product_tag):
    """retour le nova score"""
    ## nova
    nova ='XXX'
    if product_tag:
        nova_nb =  product_tag.find_all('a',href='/nova')
        if len(nova_nb) > 1:
            nova_nb =nova_nb[1]
            valeur = nova_nb.find('img').attrs['alt'].split(' - ')[0]
            try:
                pattern = re.compile(r'[+-]?([0-9]*[.])?[0-9]+$')
                test = re.match(pattern, valeur)
                if test:
                    nova = int(valeur) if valeur.isdigit() else 'XXX'
                else:
                    nova ='XXX'
            except:
                nova ='XXX'
    return nova

def recuperer_eco_score(product_tag):
    """Retour l'eco_score"""
    ## score 
    eco_score ='XXX'
    if product_tag:
        b_eco_score_nb =  product_tag.find_all('a',href='/ecoscore')
        if b_eco_score_nb:
            eco_score_img = b_eco_score_nb[-1].find('img')
            if eco_score_img is not None:
                eco_score = eco_score_img.attrs['alt'].split()[-1]
    
    return eco_score

def recuperer_caracteristique(all_p_tag):
    """Retour la caracteristique du produit"""
    caracteristique='XXX'
    if all_p_tag is not None:
        for p in all_p_tag:
            if "Dénomination générique" in p.text:
                caracteristique = supprimer_espaces_carcteres_speciaux(p.text.split(':')[1])
                break    
    return caracteristique

def recuperer_quantite(all_p_tag):
    """Retour la quantité du produit"""
    quantite='XXX'
    if all_p_tag is not None:
        for p in all_p_tag:
            if "Quantité" in p.text:
                quantite = supprimer_espaces_carcteres_speciaux(p.text.split(':')[1])
                break 
    return quantite
def recuperer_conditionnement(all_p_tag):
    """Retourne le conditionnement du produit"""
    conditionnement ="XXX"
    if all_p_tag is not None:
        for p in all_p_tag:
            if 'Conditionnement' in p.text:
                conditionnement =supprimer_espaces_carcteres_speciaux(p.text.split(':')[1])
                break
    return conditionnement

def recuperer_marques(all_p_tag):
    """Retourne le conditionnement du produit"""
    marques ="XXX"
    if all_p_tag is not None:
        for p in all_p_tag:
            if 'Marques' in p.text:
                marques =supprimer_espaces_carcteres_speciaux(p.text.split(':')[1])
                break
    return marques
def recuperer_categories(all_p_tag):
    """Retourne l.a.es catégorie.s"""
    categories ='XXX'
    if all_p_tag is not None:
        for p in all_p_tag:
            if "Catégories" in p.text:
                categories = supprimer_espaces_carcteres_speciaux(p.text.split(':')[1])
                break
    return categories
def recuperer_lieu_fabrication(all_p_tag):
    """Recuperer le lieu de fabrication"""
    lieu_fabrication ='XXX'
    if all_p_tag is not None:
        for p in all_p_tag:
            if "Lieux de fabrication ou de transformation" in p.text:
                lieu_fabrication = supprimer_espaces_carcteres_speciaux(p.text.split(':')[1].strip())
                break
    return lieu_fabrication
                                                     
def recuperer_code_tracabilite(all_p_tag):
    code_tracabilite ='XXX'
    if all_p_tag is not None:
        for p in all_p_tag:
            if "Code de traçabilité" in p.text:
                code_tracabilite = supprimer_espaces_carcteres_speciaux(p.text.split(':')[1].strip())
                break
    return code_tracabilite

def recuperer_lien_produit_page_site_fabricant(all_p_tag):
    """Recupere le lien du produit sur le site du fabriquant"""
    lien_produit_page_site_fabricant='XXX'
    if all_p_tag is not None:
        for p in all_p_tag:
            if "Lien vers la page du produit sur le site officiel du fabricant" in p.text:
                lien_produit_site_fabricant_0 = p.find('a').attrs['href'] if p.find('a') is not None else 'XXX'
                lien_produit_page_site_fabricant = supprimer_espaces_carcteres_speciaux(lien_produit_site_fabricant_0)
                break
    return lien_produit_page_site_fabricant
def recuperer_magasins(all_p_tag):
    magasins ='XXX'
    if all_p_tag is not None:
        for p in all_p_tag:
            if "Magasins" in p.text:
                magasins = p.text.split(':')[1]
                break
    return magasins

def recuperer_label_certif_recompense(all_p_tag):
    """Recuperer label, certification, recompense"""
    label_certif_recompense ='XXX'
    if all_p_tag is not None:
        for p in all_p_tag:
            if "Labels, certifications, récompenses" in p.text:
                label_certif_recompense= supprimer_espaces_carcteres_speciaux(p.text.split(':')[1])
                break
    return label_certif_recompense
def recuperer_pays_de_vente(all_p_tag):
    pays_de_vente ='XXX'
    if all_p_tag is not None:
        for p in all_p_tag:
            if "Pays de vente" in p.text:
                pays_de_vente = supprimer_espaces_carcteres_speciaux(p.text.split(':')[1])
                break
    return pays_de_vente
def recuperer_origin_ingredient(ingredient_tag):
    """Recuperer l'origine des ingrédients"""
    origin ='XXX'
    if ingredient_tag:
        origin_brut= ingredient_tag.find('div', id='ecoscore_panel_origins').find('ul')
        if origin_brut:
            origin =supprimer_espaces_carcteres_speciaux(origin_brut.text.replace('\n',','))
    return origin

def recuperer_analyse(analyse_tag):
    """Recuperer les analyses"""
    analyse ='XXX'
    if analyse_tag:
        analyse_brut = analyse_tag.find('p',id='ingredients_analysis_ingredients_text')
        if analyse_brut:
            analyse = supprimer_espaces_carcteres_speciaux(analyse_brut.text)
    return analyse
def recuperer_additifs(additif_tag):
    """Recuperer les additifs du produits"""
    additifs = 'XXX'
    if additif_tag:
        additifs_brut = additif_tag[0].find('ul') 
        if additifs_brut:
            additifs = supprimer_espaces_carcteres_speciaux(additifs_brut.text)
    return additifs

def recuperer_huile_de_palme(huile_palme_tag):
    """Recuperer huile de palme features"""
    huile_palme='XXX'
    if huile_palme_tag:
        huile_palme_brut = huile_palme_tag[1].find('ul')
        if huile_palme_brut:
            huile_palme = supprimer_espaces_carcteres_speciaux(huile_palme_brut.text)
            return 1
    else:
        return 0
    
    return huile_palme
def recuperer_matiere_grasse(matiere_grasse_tag):
    """Recuperer la matiere grasse du produit"""
    matiere_grasse ='XXX'
    if matiere_grasse_tag:
        matiere_grasse_text = matiere_grasse_tag.find('td', class_='nutriment_value')
        if matiere_grasse_text:
            matiere_grasse =matiere_grasse_text.text.strip().split()[0].replace(',','.')
            matiere_grasse =convertible(matiere_grasse) if (matiere_grasse_text !='XXX') and (matiere_grasse[0] not in['?']) else 'XXX' ### 4.9 g
    #print(matiere_grasse)
    return matiere_grasse

def recuperer_acide_gras(acide_gras_tag):
    """Recuperer l'acide gras """
    Acide_Gras ='XXX'
    if acide_gras_tag:
        acide_gras_brut = acide_gras_tag.find('td',class_='nutriment_value')
        if acide_gras_brut:
            acide_gras_text = supprimer_espaces_carcteres_speciaux(acide_gras_brut.text) 
            Acide_Gras = convertible(acide_gras_text.split()[0].replace(',','.'))
    #print("Acide Gras:",Acide_Gras)
    return Acide_Gras
def recuperer_sucre(sucre_tag):
    """Recupere la quantité de sucrre"""
    sucre ='XXX'
    if sucre_tag:
        sucre_text = sucre_tag.find('td', class_='nutriment_value')
        if sucre_text:
            sucre = convertible(sucre_text.text.strip().split()[0].replace(',','.'))
    #print(sucre)
    return sucre
def recuperer_sel(sel_tag):
    """Recupere la quantité de sel"""
    sel='XXX'
    if sel_tag:
        sel_text = sel_tag.find('td', class_='nutriment_value')
        if sel_text:
            sel = convertible(sel_text.text.strip().split()[0].replace(',','.'))
    return sel

def recuperer_comparaison(comparaison_tag):
    """Retourne la catégorie de comparaison"""
    comparaison ='XXX'
    if comparaison_tag:
        comparaison = supprimer_espaces_carcteres_speciaux(comparaison_tag.text)
    return comparaison

def recuperer_energie_kj(energie_kj_tag):
    """Retourne l'enrgie en kj"""
    energie_kj='XXX'
    if energie_kj_tag:
        energie_kj_text =energie_kj_tag.find('td',class_='nutriment_value')
        if energie_kj_text:
            energie_kj =convertible(energie_kj_text.text.strip().split('kj')[0].replace(",",'').replace(" ","").replace(u'\xa0',''))
    return energie_kj

def recuperer_energie_kcal(energie_kcal_tag):
    """retourne l'energie en kcal"""
    energie_kcal ='XXX'
    if energie_kcal_tag:
        energie_kcal_text =energie_kcal_tag.find('td',class_='nutriment_value')
        if energie_kcal_text:
            energie_kcal =convertible(energie_kcal_text.text.strip().split('kcal')[0].replace(",",'').replace(" ","").replace(u'\xa0',''))
    
    return energie_kcal

def recuperer_les_allgergies(allergie_tag):
    #print(allergie_tag)
    allergies=''
    if allergie_tag and allergie_tag is not None:
        for alg_tag in allergie_tag:
            if allergies =='':
                allergies = alg_tag.text.lower()
            else:
                allergies = allergies +','+ alg_tag.text.lower()
    else:
        allergies= 'NaN'
    return allergies

def get_data(url_product):
    """ Recupération des informations essentielles du produit du lien en paramètre 
    @url_product : string
    @return data: list
    """
    ## recuperer la reponse http
    page =  requests.get(url_product)
   
    ## transformer au format html
    bs = BeautifulSoup(page.text, 'lxml')
    
    ## detecter la balise div avec itemtype non vide
    product = bs.find("div",itemtype =re.compile(r"\w"))
    
    ##Nom du produit
    
    nom_produit= recuperer_nom(product)
    ## code barre

    code_barre = recuperer_code_barre(product)
   
    ## nutriscore
    nutri_score =  recuperer_nutriscore(product)
    
    ## nova
    nova = recuperer_nova(product)
    
    ## eco score
    eco_score = recuperer_eco_score(product)
    
    ## les balises p sans classe successifs et dont la reconnaissance se fait par le texte 
    ## i.e Quantité: pour la valeur quantite
    all_p = None
    if product is not None :
        div_paragraph = product.find('div', class_='medium-12 large-8 xlarge-8 xxlarge-8 columns')
        if div_paragraph is not None:
            all_p = div_paragraph.find_all('p')
    ## caracteristique
    caracteristique =recuperer_caracteristique(all_p)
    ## quantite
    quantite = recuperer_quantite(all_p)
    ## conditionnement
    conditionnement = recuperer_conditionnement(all_p)
    ## marques
    marques = recuperer_marques(all_p)
    ## categories
    categories = recuperer_categories(all_p)
    ## label, certifications, recompenses
    label_certif_recompense =recuperer_label_certif_recompense(all_p)
    ## lieu de fabrication
    lieu_fabrication = recuperer_lieu_fabrication(all_p)
    ## code de tracabilité
    code_tracabilite =recuperer_code_tracabilite(all_p)
    ## lien url du produit sur le site du fabricant 
    lien_produit_page_site_fabricant = recuperer_lien_produit_page_site_fabricant(all_p)
    ## les magasins vente du produit
    magasins = recuperer_magasins(all_p)
    ## les pays où est vendu le produit
    pays_de_vente = recuperer_pays_de_vente(all_p)
    ## Origine des ingredients
    ingredient_tag = bs.find('div',id='ecoscore_drop')
    origin = recuperer_origin_ingredient(ingredient_tag)
    ## l'analyse des ingredients
    analyse_tag = bs.find("div", id='ingredient_analysis_drop')
    analyse = recuperer_analyse(analyse_tag)
    ## les additifs
    additifs_tag = bs.find_all('div', class_='medium-6 columns')
    additifs= recuperer_additifs(additifs_tag)
    ## si existance d'huile de palme
    huile_palme_tag = bs.find_all('div', class_='medium-6 columns')
    huile_palme= recuperer_huile_de_palme(huile_palme_tag)
    ## quantité de matiere grasse
    matiere_grasse_tag = bs.find('tr', id ='nutriment_fat_tr')
    Matiere_grasse = recuperer_matiere_grasse(matiere_grasse_tag)
    ## acide gras
    acide_gras_tag = bs.find('tr',id='nutriment_saturated-fat_tr')
    Acide_Gras = recuperer_acide_gras(acide_gras_tag)
    ## sucre 
    sucre_tag = bs.find('tr',id ='nutriment_sugars_tr')
    Sucre = recuperer_sucre(sucre_tag)
    ## sel
    sel_tag = bs.find('tr',id='nutriment_salt_tr')
    Sel = recuperer_sel(sel_tag)
    ## catégories de comparaisons
    comparaison_tag = bs.find('label', style='display:inline;font-size:1rem;')
    comparaison = recuperer_comparaison(comparaison_tag)
    ## energie en kj
    energie_kj_tag = bs.find('tr',id='nutriment_energy_tr')
    energie_kj = recuperer_energie_kj(energie_kj_tag)
    ## energie en kcal
    energie_kcal_tag = bs.find('tr', id='nutriment_energy-kcal_tr')
    nbre_kcal = recuperer_energie_kcal(energie_kcal_tag)
    
    ## glutten et allergiogene
    tag_well_knows = bs.find_all("a",attrs={'class':'tag well_known'})
    allergie_tag =[allergie for allergie in tag_well_knows if ('allergene' in allergie.attrs['href'])] 
    allergies = recuperer_les_allgergies(allergie_tag)
    ## les données sont mises en liste et renvoyées
    data = [
        nom_produit, code_barre, nutri_score, nova, eco_score,caracteristique, 
        quantite, conditionnement,marques, categories, label_certif_recompense,
        lieu_fabrication,code_tracabilite, lien_produit_page_site_fabricant,magasins,pays_de_vente,origin,analyse,additifs,
        huile_palme,Matiere_grasse,Acide_Gras,Sucre,Sel,comparaison,
        nbre_kcal,energie_kj, allergies
    ]
    return data

In [29]:
urls_produits.index("https://fr.openfoodfacts.org/produit/3274080005003/cristaline")

0

## <div class='text text-primary' style='background:#0350F0; color:white; font-size:15px; padding:20px'>Étape 4 Récuperation des données et export en CSV</div>
<hr>
> Dans cette partie, nous allons extraire les données et les stocker dans un fichier au format csv , au fur et à mesure que l'extraction avance. 

> <div class='text text-primary' style='background:#000000; color:white; font-size:15px; padding:20px'> <b>À TOUT MOMENT ON PEUT INTERROMPRE LE PROGRAMME ET LE REPRENDRE SANS PERTE DE DONNÉES NI DEVOIR RECOMMENCER </b></div>

In [ ]:

off_data = [] ## list des nos données extraites
i=0 ## compteur d'enregistrement

## les colonnes du DF
cols = ['nom_produit', 'code_bar', 'nutri_score', 'nova', 'eco_score','caracteristique', 
        'quantite', 'conditionnement','marques', 'categories', 'label_certif_recompense',
        'lieu_fabrication','code_tracabilite', 'lien_produit_page_site_fabricant','magasin','pays_de_vente','origin','analyse','additifs',
        'huile_palme','matiere_grasse','acide_gras','sucre','sel','comparaison',
        'nbre_kcal','energie_kcal','allergies']

## Verifier si nous avons un fichier qui sauvegarde l'état davancement du sauvegadre des données

print("Debut de scraping..")

## colonne de du fichier STATUS_FILE
col_status = ['last_url']

## statut de lecture du dataset csv : par défaut à False
readFile = False
## index est utlisé pour recevoir l'index du lien du dernier produit récupérer afin de rebondir au suivant lien
index = -1
if path.exists(STATUS_FILE):
    ## lire le fichier status
    df_status = pd.read_csv(STATUS_FILE)
    ## recuperer le dernier lien traité
    last_url = list(df_status['last_url'])[0]
    ## recuperer l'index dans les urls des produits
    try:
        index = urls_produits.index(last_url)
    except ValueError:
        index =-1

## A partir de l'index de l'url du fichier status . la boucle for debutera à l'index suivant 
## sinon .. on debut à l'index 0
if index != -1:
    ## concatenation de Dataframe
    readFile = True
    i = index
    
if readFile:
    for url in urls_produits[index+1:]:
        info_product = get_data(url)
        off_data.append(info_product)
        i=i+1
        #time.sleep(0.01)
        if i%100 ==0:
            print("Nombre d'execution i=",i)
            ## onn lit le fichier sauvegarder
            DF = pd.read_csv('OFF_DATA_.csv', engine='python')
            ## df avec les 100 derniers enregsistrements
            df = pd.DataFrame(off_data[-100:],columns=cols)
            #df.to_csv(DATA_FILE, encoding='utf-8', index=False)
            ## on stocke les données dans le fichier au format csv
            ## si données deja enregistrées alors on concatene avec le nouveau dataframe df

            newDF = pd.concat([DF, df], ignore_index=True)
            newDF.drop_duplicates(ignore_index=True, inplace=True)
            newDF.to_csv('OFF_DATA_.csv', encoding='utf-8', index=False)
            ## on stocke le dernier lien traité dans un fichier pour une prochaine relance (à programmer)
            df_status = pd.DataFrame([url],columns = col_status)

            df_status.to_csv(STATUS_FILE,encoding='utf-8', index=False)
        
else:
    for url in urls_produits:
        info_product = get_data(url)
        off_data.append(info_product)
        i=i+1
        #time.sleep(0.01)
        if i%100 ==0:
            print("Nombre d'execution i=",i)
            df = pd.DataFrame(off_data,columns=cols)
            #df.to_csv(DATA_FILE, encoding='utf-8', index=False)
            ## on stocke les données dans le fichier au format csv

            ## si données deja enregistrées alors on concatene avec le nouveau dataframe df

            df.to_csv('OFF_DATA_.csv', encoding='utf-8', index=False)
            ## on stocke le dernier lien traité dans un fichier pour une prochaine relance (à programmer)
            df_status = pd.DataFrame([url],columns = col_status)

            df_status.to_csv(STATUS_FILE,encoding='utf-8', index=False)
        

Made with love.. Amadou